<a href="https://colab.research.google.com/github/mudogruer/SLMs/blob/main/Llama-2/llama_2_13b_load_adapters_with_hf_peft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U peft bitsandbytes evaluate bert-score

In [ ]:
from peft import PeftConfig

In [ ]:
peft_model_id = "mudogruer/Llama-2-13b-hf-SciQ-20pc"

In [ ]:
config = PeftConfig.from_pretrained(peft_model_id)

adapter_config.json:   0%|          | 0.00/697 [00:00<?, ?B/s]

In [ ]:
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM

In [ ]:
from huggingface_hub import login
login()

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict = True,
    load_in_4bit = True,
    device_map = "auto"

)

config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    config.base_model_name_or_path,
    padding_size = "right",
    add_eos_token = True,

)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
from peft import PeftModel

fine_tuned_model = PeftModel.from_pretrained(
    model,
    peft_model_id
)

adapter_model.safetensors:   0%|          | 0.00/782M [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline, logging

2024-02-22 19:26:51.212012: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-22 19:26:51.212112: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-22 19:26:51.362987: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
logging.set_verbosity(logging.CRITICAL)

In [ ]:
pipe = pipeline(
    task = "text-generation",
    model = fine_tuned_model,
    tokenizer = tokenizer,
    eos_token_id = model.config.eos_token_id,
    max_new_tokens = 30,

)

In [ ]:
prompt = "Which molecule in the air is the most?"
result = pipe(f"<s>[INST]{prompt}[/INST]")

/opt/conda/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


In [ ]:
result

[{'generated_text': '<s>[INST]Which molecule in the air is the most?[/INST] nitrogen'}]

In [ ]:
def Sequence(promt):
    sequence = pipe(
    prompt,
    do_sample = True,
    max_new_tokens = 30,
    temperature = 0.1,
    top_k = 2,
    top_p = 1.0,
    pad_token_id=tokenizer.eos_token_id
    )
    return sequence

In [ ]:
prompt = "<s>[INST]"+"Which is the heaviest element?"+"[/INST]"
print(Sequence(prompt)[0]['generated_text'])

<s>[INST]Which is the heaviest element?[/INST] oxygen


In [ ]:
from datasets import load_dataset

dataset = load_dataset("sciq")

Generating train split:   0%|          | 0/11679 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset sciq downloaded and prepared to /root/.cache/huggingface/datasets/sciq/default/0.1.0/50e5c6e3795b55463819d399ec417bfd4c3c621105e00295ddb5f3633d708493. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset_validation = load_dataset('sciq',split='validation[:20%]') #first %20 of validation data

In [ ]:
dataset_validation

Dataset({
    features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
    num_rows: 200
})

In [ ]:
answers = []
for question in dataset_validation["question"]:
    prompt = "<s>[INST]"+question+"[/INST]:"
    output_sequences = Sequence(prompt)[0]['generated_text']
    question_and_answer = output_sequences.split("[/INST]")
    answer = question_and_answer[1]
    answers.append(answer)

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
from evaluate import load
import numpy as np
from bert_score import score

bertscore = load("bertscore")
predictions = answers
references = dataset_validation["correct_answer"]
results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
print("precision: ",round(np.mean(list(results["precision"])),5))
print("recall: ",round(np.mean(list(results["recall"])),5))
print("f1: ",round(np.mean(list(results["f1"])),5))

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

precision:  0.66426
recall:  0.81022
f1:  0.72845
